In [1]:
import numpy as np

"""STVM 계산(3차로 점선)"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

Q_critical = 2200

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1,
    "w7" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])["v[km/h]"]
                .agg(['mean', 'count'])
                .reset_index()
    )
    speed_mean_df["v_next"] = speed_mean_df.groupby("TimeGroup")["mean"].shift(-1)
    speed_mean_df["delta_v"] = (speed_mean_df["v_next"] - speed_mean_df["mean"]) / speed_mean_df["mean"]
    display(speed_mean_df)
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(density = df["count"] * 12 / df["mean"])
    return density_mean_df
###################################################################################################################


folder_path = r"C:\VISSIM_Workspace\network_new\network_version2\network_version2.3\network_version2.3.0\network03_mer"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
mer_file = mer_list[0]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()




# 원시 데이터 기초 가공처리
with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()
        # 데이터가 시작하는 인덱스 찾기
        data_start_idx = None

        for j, line in enumerate(lines):
            if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                data_start_idx = j
                break

        # 데이터프레임 생성
        if data_start_idx is not None:

            # 컬럼명 추출 및 공백 제거
            columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

            # 데이터 부분 추출 및 가공
            data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
            data = [line.strip().split(";") for line in data_lines if line.strip()]

            # 데이터프레임 생성
            df = pd.DataFrame(data, columns=columns)

            # 컬럼 내부 데이터 정수형 변환
            df = df.apply(pd.to_numeric, errors="coerce")

            original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


            #불필요 컬럼 제거
            original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

            original_df["New_Measurement"] = original_df["Measurem."] % 1000

            bins = np.arange(start_interval, end_interval+1, 300)
            labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

            # 구간 나누기 및 컬럼 추가
            original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

            # 평균속도
            speed_df = speed_mean(original_df)

            # 밀도
            density_df = density_mean(speed_df)
            display(density_df)

            #print(len(original_df[(original_df["TimeGroup"] =="5100~5400") & (original_df["New_Measurement"] == 116)]))


,TimeGroup,New_Measurement,mean,count,v_next,delta_v
0,1800~2100,1,106.585333,150,106.146667,-0.004116
1,1800~2100,2,106.146667,150,105.774667,-0.003505
2,1800~2100,3,105.774667,150,105.313907,-0.004356
3,1800~2100,4,105.313907,151,104.994000,-0.003038
4,1800~2100,5,104.994000,150,104.743137,-0.002389
...,...,...,...,...,...,...
1423,5100~5400,116,106.263265,147,106.032000,-0.002176
1424,5100~5400,117,106.032000,150,105.902013,-0.001226
1425,5100~5400,118,105.902013,149,105.826316,-0.000715
1426,5100~5400,119,105.826316,152,105.905921,0.000752


,TimeGroup,New_Measurement,mean,count,v_next,delta_v,density
0,1800~2100,1,106.585333,150,106.146667,-0.004116,16.887877
1,1800~2100,2,106.146667,150,105.774667,-0.003505,16.957669
2,1800~2100,3,105.774667,150,105.313907,-0.004356,17.017307
3,1800~2100,4,105.313907,151,104.994000,-0.003038,17.205705
4,1800~2100,5,104.994000,150,104.743137,-0.002389,17.143837
...,...,...,...,...,...,...,...
1423,5100~5400,116,106.263265,147,106.032000,-0.002176,16.600280
1424,5100~5400,117,106.032000,150,105.902013,-0.001226,16.976007
1425,5100~5400,118,105.902013,149,105.826316,-0.000715,16.883532
1426,5100~5400,119,105.826316,152,105.905921,0.000752,17.235789
